# Question: Does the company treat its employees well? 

This section relates to cultural aspects of the companies being examined, including the chief concern: how employees are treated. We also consider worker safety; fair pay and benefits; opportunities for development, training, and advancement; and other aspects that impact the company's workers.

Employees are the lifeblood of a company. Happy, healthy, and valued employees are more willing and able to do higher-quality work, be enthusiastic brand evangelists, unleash their creativity to invent better services and solutions, and innovate to improve the company. Employees should be viewed as valuable assets to invest in continually, not expendable "resources" or drags on the bottom line. (The "bottom line" is net profit -- a company's income after all expenses have been deducted from revenue.)

Companies that excel at engaging their employees actually achieve per-share earnings growth more than four times that of their rivals, according to Gallup. Compared to companies in the bottom quartile, the top-quartile companies (based on employee engagement) generate higher customer engagement, higher productivity, better retention, fewer accidents, and 21% higher profitability.

Even though Wall Street tends to cheer when companies lay off workers, high employee turnover is actually an expense to be avoided. Not only is it a financial cost -- think about severance packages, and the costs of recruiting and training new employees as well as retraining remaining workers -- but the loss of intellectual capital is also a poor outcome for employers.

Company websites and sustainability reports can help you assess this factor. Also look for publications from organizations that rate companies on worker treatment, such as Fortune's annual list of "100 Best Companies to Work For" and Forbes' "Just 100" list.

We also look for negative elements, like shoddy employment treatment; contentious relationships with unions; lawsuits or controversies about discrimination; harassment or wage theft; and other behavior that indicates poor employee relations, like serial layoffs or constant restructuring. These kinds of red flags might disqualify a company for inclusion in our ESG portfolio.

# Data Sources

Possible Sites to Aggregate Employee Reviews:
- Glassdoor 
- Indeed
- Vault 
- CareerBliss
- Kununu
- JobAdvisor
- Ratemyemployer
- TheJobCrowd
- LookBeforeYouLeap
- Comparably
- Yelp

# Imports

In [1]:
# All necessary imports
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests
import pandas as pd
pd.set_option("display.max_columns", 50)
import pickle
import newspaper
import re
from functools import reduce

# Data Collection

For the time being, only Apple will be used. Once the process is finished for a single stock, we can run back through to get a bunch for the industry. 

In [25]:
companies = ['VOXX International Corporation',
             'Apple Inc',
             'Best Buy Co Inc',
             'GameStop Corporation',
             'Sony Corporation',
             'Activision Blizzard Inc',
             'Netflix, Inc',
             'Roku, Inc']

## Indeed

In [40]:
def get_indeed_reviews(companies):
    
    # Create empty dataframe
    indeed_reviews = pd.DataFrame(columns=['company','work_life_balance','pay_and_benefits','job_security_and_advancement','management','culture','composite_score'])
    
    # Set path to chromedriver
    PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
    # Define options 
    options = Options()
    # Remove pop up window
    options.add_argument("--headless")
    # Define driver
    driver = webdriver.Chrome(PATH, options=options)
    # # Define driver
    # driver = webdriver.Chrome(PATH)
    driver.set_window_size(1080,800)
    # Define url
    url= "https://www.indeed.com/companies?from=gnav-acme--discovery-webapp"
    # Get website
    driver.get(url)
    
    for company in companies:
        try:
            # Find search bar
            search_bar = driver.find_element_by_xpath('//*[@id="exploreCompaniesWhat"]')
            # Clear search bar
            search_bar.clear()
            # Enter company name into search bar
            search_bar.send_keys(company)
            # Search company
            search_bar.send_keys(Keys.ENTER)
            search_bar.send_keys(Keys.ENTER)
            time.sleep(3)
            try:
                # Get company
                driver.find_element_by_xpath('//*[@id="cmp-discovery"]/div[2]/div/div[2]/div/div[1]/div[2]').click()
            except:
                time.sleep(6)
                exit_button = driver.find_element_by_xpath('//*[@id="popover-x"]/button')
                exit_button.click()
            time.sleep(2)
            # Define variables for employee ratings
            try:
                work_life_balance = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[1]/a/span[1]').text)
            except:
                work_life_balance = 'NaN'
            try:
                pay_and_benefits = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[2]/a/span[1]').text)
            except:
                pay_and_benefits = 'NaN'
            try:
                job_security_and_advancement = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[3]/a/span[1]').text)
            except:
                job_security_and_advancement = 'NaN'
            try:
                management = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[4]/a/span[1]').text)
            except:
                management = 'NaN'
            try:
                culture = float(driver.find_element_by_xpath('//*[@id="cmp-container"]/div/div[1]/main/div/div[2]/div[1]/div[2]/div[2]/div[5]/a/span[1]').text)
            except:
                culture = 'NaN'
            try:
                composite_score = (work_life_balance+pay_and_benefits+job_security_and_advancement+management+culture) / 25
            except:
                composite_score = 'NaN'
            # Append new data to dataframe
            indeed_reviews = indeed_reviews.append({'company': company,
                                                    'work_life_balance': work_life_balance,
                                                    'pay_and_benefits': pay_and_benefits,
                                                    'job_security_and_advancement': job_security_and_advancement,
                                                    'management': management,
                                                    'culture': culture,
                                                    'composite_score': composite_score}, ignore_index=True)
        
            time.sleep(1)
            driver.find_element_by_link_text('Company Reviews').click()
            time.sleep(2)
            time.sleep(1)
            
        except:
            pass
    
    return indeed_reviews
    

In [41]:
indeed_reviews = get_indeed_reviews(companies)

In [42]:
indeed_reviews

,company,work_life_balance,pay_and_benefits,job_security_and_advancement,management,culture,composite_score
0,VOXX International Corporation,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple Inc,3.8,4.1,3.7,3.7,4.1,0.776
2,Best Buy Co Inc,3.7,3.5,3.3,3.4,3.8,0.708
3,GameStop Corporation,3.4,2.8,2.9,3.4,3.5,0.64
4,Sony Corporation,3.9,3.8,3.6,3.7,3.9,0.756
5,Activision Blizzard Inc,3.1,3.2,2.8,3.5,3.8,0.656
6,"Netflix, Inc",NaN,4,3.2,NaN,NaN,NaN
7,"Roku, Inc",3.1,NaN,2.3,2.5,2.6,NaN


## Vault

Doesn't have uppers and downers for all companies

In [40]:
# def get_vault_reviews(companies):
    
#     vault_reviews = pd.DataFrame(columns=['company','uppers','downers'])
    
#     # Set path to chromedriver
#     PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
#     # Define options 
#     options = Options()
#     # Remove pop up window
#     options.add_argument("disable-infobars")
# #     options.add_argument("--headless")
#     # Define driver
#     driver = webdriver.Chrome(PATH, options=options)
#     # # Define driver
#     # driver = webdriver.Chrome(PATH)
#     driver.set_window_size(1080,800)
#     # Define url
#     url= "https://www.vault.com/"
#     # Get website
#     driver.get(url)
#     time.sleep(2)

#     for company in companies:
#         # Find search bar
#         search_bar = driver.find_element_by_xpath('//*[@id="HeroSearchBox"]')
#         # Clear search bar
#         search_bar.clear()
#         # Enter company name into search bar
#         search_bar.send_keys(company)
#         time.sleep(3)
#         # Search company
#         search_bar.send_keys(Keys.ENTER)
#         time.sleep(4)
#         # Find first company listed
#         likely_company = driver.find_element_by_xpath('//*[@id="feed-article-1"]/div/h2/a')
#         # Click on company
#         likely_company.click()
#         time.sleep(3)
#         # Get company positives
#         uppers = (driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/div[2]/div[1]/div[1]').text).replace('\n',' ').lstrip('Uppers').strip()
#         # Get company negatives
#         downers = (driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/div[2]/div[1]/div[2]').text).replace('\n',' ').lstrip('Downers').strip()
        
#         # Append new data to dataframe
#         vault_reviews = vault_reviews.append({'company': company,
#                                                 'uppers': uppers,
#                                                 'downers': downers}, ignore_index=True)
        
#         time.sleep(2)
#         driver.back()
#         time.sleep(1)
#         driver.back()
#         time.sleep(3)
    
#     vault_reviews.uppers = vault_reviews.uppers.apply(lambda x: x.lower())
#     vault_reviews.downers = vault_reviews.downers.apply(lambda x: x.lower())
        
#     return vault_reviews   

In [41]:
# vault_reviews = get_vault_reviews(companies)

# Career Bliss 

In [48]:
# def get_careerbliss_reviews(companies):
    
#     careerbliss_reviews = pd.DataFrame(columns=['company','company_culture','coworkers','rewards','way_you_work','growth_opp','person_you_work_for','support','work_setting'])
    
#     # Set path to chromedriver
#     PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
#     # Define options 
#     options = Options()
#     # Remove pop up window
#     options.add_argument("disable-infobars")
#     options.add_argument("--headless")
#     # Define driver
#     driver = webdriver.Chrome(PATH, options=options)
#     # # Define driver
#     # driver = webdriver.Chrome(PATH)
#     driver.set_window_size(1080,800)
#     # Define url
#     url= "https://www.careerbliss.com/reviews/"
#     # Get website
#     driver.get(url)
#     time.sleep(2)
#     for company in companies:
#         try:
#             # Find search bar
#             search_bar = driver.find_element_by_xpath('//*[@id="search-q"]')
#             # Clear search bar
#             search_bar.clear()
#             # Enter company name into search bar
#             search_bar.send_keys(company)
#             time.sleep(3)
#             # Search company
#             search_bar.send_keys(Keys.ENTER)
#             time.sleep(4)
#             try:
#                 # Find first company listed
#                 likely_company = driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div[1]/div[3]/div[2]/div[2]/div[2]/div/div/span/a')
#                 # Click on company
#                 likely_company.click()
#             except:
#                 driver.refresh()
#                 time.sleep(3)
#                 # Find first company listed
#                 likely_company = driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div[1]/div[3]/div[2]/div[2]/div[2]/div/div/span/a')
#                 # Click on company
#                 likely_company.click()
#             time.sleep(3)
#             company_culture = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[1]/div[1]/div/span[3]').text)
#             coworkers = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[2]/div[1]/div/span[3]').text)
#             rewards = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[3]/div[1]/div/span[3]').text)
#             way_you_work = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[4]/div[1]/div/span[3]').text)
#             growth_opp = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[1]/div[2]/div/span[3]').text)
#             person_you_work_for = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[2]/div[2]/div/span[3]').text)
#             support = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[3]/div[2]/div/span[3]').text)
#             work_setting = float(driver.find_element_by_xpath('//*[@id="bodyContainer"]/div/div/div[6]/div[4]/div[2]/div/span[3]').text)
#         except:
#             pass
#         time.sleep(2)
#         driver.back()
#         time.sleep(2)
#         driver.back()
#         time.sleep(2)
        
#         careerbliss_reviews = careerbliss_reviews.append({'company': company,
#                                                           'company_culture': company_culture,
#                                                           'coworkers': coworkers,
#                                                           'rewards': rewards,
#                                                           'way_you_work': way_you_work,
#                                                           'growth_opp': growth_opp,
#                                                           'person_you_work_for': person_you_work_for,
#                                                           'support': support,
#                                                           'work_setting': work_setting}, ignore_index=True)
    
#     return careerbliss_reviews

In [49]:
careerbliss_reviews = get_careerbliss_reviews(companies)

In [50]:
careerbliss_reviews

,company,company_culture,coworkers,rewards,way_you_work,growth_opp,person_you_work_for,support,work_setting
0,Apple Inc,4.1,4.6,4.0,4.2,3.7,4.3,4.3,4.1
1,Best Buy Co Inc,3.7,4.4,3.4,4.0,3.2,4.1,3.8,4.0
2,GameStop Corporation,3.6,4.2,3.1,4.0,3.0,4.0,3.6,3.6
3,Sony Corporation,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0
4,Activision Blizzard Inc,3.2,3.9,3.5,3.2,2.1,3.4,3.4,3.2
5,"Netflix, Inc",4.4,4.1,4.3,4.0,3.5,4.2,4.2,4.2
6,"Roku, Inc",4.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0
7,VOXX International Corporation,4.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0


# Comparably

In [57]:
def get_comparably_reviews(companies):
    
    comparably_reviews = pd.DataFrame(columns=['company','culture','ceo_score','net_promoter_scale','perks_and_benefits','outlook','executive_team','work_culture','compensation','leadership','diversity','team','happiness','environment','gender','manager','retention','meetings','professional_development','office_culture'])
    
    # Set path to chromedriver
    PATH = "/Users/MichaelWirtz/Desktop/pathfile/chromedriver_2"
    # Define options 
    options = Options()
    # Remove pop up window
    options.add_argument("disable-infobars")
#     options.add_argument("--headless")
    # Define driver
    driver = webdriver.Chrome(PATH, options=options)
    # # Define driver
    # driver = webdriver.Chrome(PATH)
    driver.set_window_size(1080,800)
    # Define url
    url= "https://www.comparably.com/"
    # Get website
    driver.get(url)
    time.sleep(2)
    for company in companies:
        try:
            # Find search bar
            search_bar = driver.find_element_by_xpath('//*[@id="bodyContent"]/div[2]/div/div/div/div/form/div/div/input')
            # Clear search bar
            search_bar.clear()
            # Enter company name into search bar
            search_bar.send_keys(company)
            time.sleep(1)
            # Search company
            search_bar.send_keys(Keys.ENTER)
            time.sleep(4)
            # Find first company listed
            likely_company = driver.find_element_by_xpath('//*[@id="bodyContent"]/div[2]/div/div[2]/div[1]/div/div[2]/a[1]/div/div/div[1]')
            # Click on company
            likely_company.click()
            time.sleep(2)
            try:
                driver.find_element_by_xpath('/html/body/div[4]/a').click()
            except:
                pass
            try:
                culture = float(driver.find_element_by_xpath('//*[@id="bodyContent"]/div[2]/div[4]/div/div/div/div/div[1]/div[1]/a/div[2]/div[1]/div[2]/b').text)
            except:
                culture - 'NaN'
            try:
                ceo_score = float(driver.find_element_by_xpath('//*[@id="bodyContent"]/div[2]/div[4]/div/div/div/div/div[1]/div[3]/a/div[4]/span/span[1]/span').text)
            except:
                ceo_score = 'NaN'
            try:
                net_promoter_scale = float(driver.find_element_by_xpath('//*[@id="bodyContent"]/div[2]/div[4]/div/div/div/div/div[2]/div[2]/a[3]/div[2]/div/div[2]/div[1]').text)
            except:
                net_promoter_scale = 'NaN'
            time.sleep(2)
            try:
                driver.find_element_by_link_text('Culture').click()
                time.sleep(2)
                # Getting page content
                content = driver.page_source.encode('utf-8').strip()
                # Getting page content in html
                soup = BeautifulSoup(content,"html.parser")
                # perks_and_benefits = soup.find_all("a", href=re.compile("perks-and-benefits"))
                table = soup.find(class_="gs-row offset cppCultureGrades-Grades")
                ratings_squares = table.find_all(class_="gs-col gs-col-1-2")
                for i in range(0,18):
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Perks And Benefits':
                        perks_and_benefits = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Outlook':
                        outlook = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Executive Team':
                        executive_team = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Work Culture':
                        work_culture = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Compensation':
                        compensation = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Leadership':
                        leadership = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Diversity':
                        diversity = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Team':
                        team = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Happiness':
                        happiness = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Environment':
                        environment = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Gender':
                        gender = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Manager':
                        manager = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Retention':
                        retention = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Meetings':
                        meetings = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Professional Development':
                        professional_development = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
                    if ratings_squares[i].find(class_ = 'section-subtitle').text == 'Office Culture':
                        office_culture = float(ratings_squares[i].find(class_ = 'numberGrade-score').text)
            except:
                perks_and_benefits = 'NaN'
                outlook = 'NaN'
                executive_team = 'NaN'
                work_culture = 'NaN'
                compensation = 'NaN'
                leadership = 'NaN'
                diversity = 'NaN'
                team = 'NaN'
                happiness = 'NaN'
                environment = 'NaN'
                gender = 'NaN'
                manager = 'NaN'
                retention = 'NaN'
                meetings = 'NaN'
                professional_development = 'NaN'
                office_culture = 'NaN'

            comparably_reviews = comparably_reviews.append({'company': company,
                                                            'culture': culture,
                                                            'ceo_score': ceo_score,
                                                            'net_promoter_scale': net_promoter_scale,
                                                            'perks_and_benefits': perks_and_benefits,
                                                            'outlook': outlook,
                                                            'executive_team': executive_team,
                                                            'work_culture': work_culture,
                                                            'compensation': compensation,
                                                            'leadership': leadership,
                                                            'diversity': diversity,
                                                            'team': team,
                                                            'happiness': happiness,
                                                            'environment': environment,
                                                            'gender': gender,
                                                            'manager': manager,
                                                            'retention': retention,
                                                            'meetings': meetings,
                                                            'professional_development': professional_development,
                                                            'office_culture': office_culture}, ignore_index=True)

        except:
            pass
        
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="body"]/header/div/div/div[1]/div/a').click()
        time.sleep(2)
        
    return comparably_reviews

In [58]:
comparably_reviews = get_comparably_reviews(companies)

In [59]:
comparably_reviews

,company,culture,ceo_score,net_promoter_scale,perks_and_benefits,outlook,executive_team,work_culture,compensation,leadership,diversity,team,happiness,environment,gender,manager,retention,meetings,professional_development,office_culture
0,Apple Inc,4.3,80.0,24.0,81.0,78.0,74.0,76.0,74.0,73.0,73.0,77.0,75.0,73.0,73.0,69.0,69.0,68.0,58.0,59.0
1,Best Buy Co Inc,4.1,77.0,21.0,70.0,74.0,72.0,75.0,70.0,73.0,70.0,78.0,72.0,74.0,73.0,73.0,72.0,66.0,61.0,60.0
2,GameStop Corporation,3.0,61.0,-17.0,57.0,62.0,55.0,66.0,58.0,57.0,64.0,72.0,61.0,70.0,65.0,60.0,64.0,60.0,54.0,54.0
3,Sony Corporation,3.8,72.0,3.0,81.0,71.0,64.0,70.0,78.0,67.0,69.0,72.0,74.0,69.0,64.0,68.0,67.0,64.0,56.0,67.0
4,"Roku, Inc",3.3,72.0,8.0,65.0,76.0,67.0,74.0,63.0,67.0,62.0,73.0,76.0,71.0,71.0,75.0,70.0,66.0,53.0,61.0


# Merging Dataframes

In [65]:
data_frames = [indeed_reviews, careerbliss_reviews, comparably_reviews]

In [66]:
question_1 = reduce(lambda  left,right: pd.merge(left,right,on=['company'],how='outer'), data_frames)

In [67]:
question_1

,company,work_life_balance,pay_and_benefits,job_security_and_advancement,management,culture_x,composite_score,company_culture,coworkers,rewards,way_you_work,growth_opp,person_you_work_for,support,work_setting,culture_y,ceo_score,net_promoter_scale,perks_and_benefits,outlook,executive_team,work_culture,compensation,leadership,diversity,team,happiness,environment,gender,manager,retention,meetings,professional_development,office_culture
0,Apple Inc,3.8,4.1,3.7,3.7,4.1,0.776,4.1,4.6,4.0,4.2,3.7,4.3,4.3,4.1,4.3,80.0,24.0,81.0,78.0,74.0,76.0,74.0,73.0,73.0,77.0,75.0,73.0,73.0,69.0,69.0,68.0,58.0,59.0
1,Best Buy Co Inc,3.7,3.5,3.3,3.4,3.8,0.708,3.7,4.4,3.4,4.0,3.2,4.1,3.8,4.0,4.1,77.0,21.0,70.0,74.0,72.0,75.0,70.0,73.0,70.0,78.0,72.0,74.0,73.0,73.0,72.0,66.0,61.0,60.0
2,GameStop Corporation,3.4,2.8,2.9,3.4,3.5,0.640,3.6,4.2,3.1,4.0,3.0,4.0,3.6,3.6,3.0,61.0,-17.0,57.0,62.0,55.0,66.0,58.0,57.0,64.0,72.0,61.0,70.0,65.0,60.0,64.0,60.0,54.0,54.0
3,Sony Corporation,3.9,3.8,3.6,3.7,3.9,0.756,5.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,3.8,72.0,3.0,81.0,71.0,64.0,70.0,78.0,67.0,69.0,72.0,74.0,69.0,64.0,68.0,67.0,64.0,56.0,67.0
4,Activision Blizzard Inc,3.1,3.2,2.8,3.5,3.8,0.656,3.2,3.9,3.5,3.2,2.1,3.4,3.4,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Netflix, Inc",NaN,NaN,NaN,NaN,NaN,NaN,4.4,4.1,4.3,4.0,3.5,4.2,4.2,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Roku, Inc",NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VOXX International Corporation,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Making Sense Out of Data

In [68]:
data = question_1

In [70]:
data['culture'] = ((data.culture_x *20) + 
                   (data.culture_y *20) + 
                   data.work_culture + 
                   data.office_culture + 
                   (data.company_culture * 20)) / 5

data['opportunity'] = ((data.job_security_and_advancement * 20) + 
                       (data.growth_opp * 20) + 
                       data.professional_development) / 3

data['leadership_and_management'] = ((data.management*20) + 
                                     (data.person_you_work_for*20) + 
                                     data.ceo_score + 
                                     data.executive_team + 
                                     data.leadership + 
                                     data.manager) / 6

data['pay_perks_and_benefits'] = ((data.work_life_balance*20) + 
                                  (data.pay_and_benefits*20) + 
                                  data.perks_and_benefits + 
                                  data.compensation + 
                                  (data.rewards*20)) / 4

data['day_to_day'] = ((data.coworkers*20) + 
                      (data.way_you_work*20) + 
                      data.team + 
                      data.meetings + 
                      (data.work_setting*20)) / 5

data['mental_health'] = ((data.support*20) +
                         data.outlook + 
                         data.happiness + 
                         data.retention) / 4

data['diversity_inclusion'] = (data.diversity + 
                               data.gender) / 2

data = data.drop(columns=['culture_x',
                          'culture_y',
                          'work_culture',
                          'office_culture',
                          'company_culture',
                          'composite_score',
                          'job_security_and_advancement',
                          'growth_opp',
                          'professional_development',
                          'management',
                          'person_you_work_for',
                          'ceo_score',
                          'executive_team',
                          'leadership',
                          'manager',
                          'work_life_balance',
                          'pay_and_benefits',
                          'perks_and_benefits',
                          'compensation',
                          'rewards',
                          'coworkers',
                          'way_you_work',
                          'team',
                          'meetings',
                          'work_setting',
                          'support',
                          'outlook',
                          'happiness',
                          'retention',
                          'diversity',
                          'gender'], axis=1)

data.rename(columns={'net_promoter_scale': 'refer_to_friend'}, inplace=True)

In [71]:
data

,company,refer_to_friend,environment,culture,opportunity,leadership_and_management,pay_perks_and_benefits,day_to_day,mental_health,diversity_inclusion
0,Apple Inc,24.0,73.0,77.0,68.666667,76.000000,98.25,80.6,77.00,73.0
1,Best Buy Co Inc,21.0,74.0,73.4,63.666667,74.166667,88.00,78.4,73.50,71.5
2,GameStop Corporation,-17.0,70.0,64.4,57.333333,63.500000,75.25,73.6,64.75,64.5
3,Sony Corporation,3.0,69.0,78.2,76.000000,74.166667,98.25,87.2,78.00,66.5
4,Activision Blizzard Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Netflix, Inc",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Roku, Inc",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VOXX International Corporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
